# Weather data fetch tests
This notebook will walk through how to get forecasted weather data for a location. 

the weather api used is from weather.gov: https://www.weather.gov/documentation/services-web-api

# the weather fetching process is:
1.  get the lat long for the sites
2.  get the grid points
3.  call the for the appropriate forecast 


# Get lat long for each site
Using google maps, I typed in the locations and then asked for the lat and long of that spot on the map. Here are the lat long coordinates for our locations
![caltech](caltech_lat_long.png)

In [1]:
import requests
import pandas as pd
import json

caltech_lat =   34.134785646454844
caltech_lon = -118.11691382579643

jpl_lat = 34.20142342818471
jpl_lon = -118.17126565774107

office_lat = 37.33680466796926
office_lon = -121.90743423142634

# get grid points
The way the weather api system works is by finding the station's grid points that are closest to the lat long coordinates. The code below gets the grid x and grid y from the lat long coordinates

In [2]:
def get_grid_points(latitude, longitude):
    url = f'https://api.weather.gov/points/{latitude},{longitude}'
    print(url)
    
    r = requests.get(url)
    
    print('status code:', r.status_code)
    payload = r.json()
    
    gridId = payload['properties']['gridId']
    gridX = payload['properties']['gridX']
    gridY = r.json()['properties']['gridY']
    return gridId, gridX, gridY
cal = get_grid_points(caltech_lat, caltech_lon)
off = get_grid_points(office_lat, office_lon)

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200


('LOX', 160, 48)

In [3]:
ct_grid, ct_grid_x, ct_grid_y = get_grid_points(caltech_lat, caltech_lon)
jpl_grid, jpl_grid_x, jpl_grid_y = get_grid_points(jpl_lat, jpl_lon)
office_grid, office_grid_x, office_grid_y = get_grid_points(office_lat, office_lon)

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/points/34.20142342818471,-118.17126565774107
status code: 200
https://api.weather.gov/points/37.33680466796926,-121.90743423142634
status code: 200


# get forecast
Once we have the office, gridx and gridy for the location, we can get three different kinds of weather forecasts

    forecast - forecast for 12h periods over the next seven days
    forecastHourly - forecast for hourly periods over the next seven days
    forecastGridData - raw forecast data over the next seven days



In [4]:
'''
def get_weather_forecast(office, gridX, gridY, forecast_type='forecast'):
    if forecast_type not in ['12_hour', '7day_hourly', '7day_grid']:
        print(f'{forecast_type} is not a valid option. please select 12_hour, 7day_hourly, or 7day_grid', )
        pass
    
    forecast_urls = {'12_hour':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast',
                     '7day_hourly':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly',
                     '7day_grid':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'}
    
    url = forecast_urls[forecast_type]
    print(url)
    r = requests.get(url)
    return r.json()
'''


"\ndef get_weather_forecast(office, gridX, gridY, forecast_type='forecast'):\n    if forecast_type not in ['12_hour', '7day_hourly', '7day_grid']:\n        print(f'{forecast_type} is not a valid option. please select 12_hour, 7day_hourly, or 7day_grid', )\n        pass\n    \n    forecast_urls = {'12_hour':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast',\n                     '7day_hourly':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly',\n                     '7day_grid':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'}\n    \n    url = forecast_urls[forecast_type]\n    print(url)\n    r = requests.get(url)\n    return r.json()\n"

In [5]:
def get_weather_forecast(office, gridX, gridY):
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast'
    print(url)
    r = requests.get(url)
    return r.json()

In [6]:
def get_hourly_weather_forecast(office, gridX, gridY):
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly'
    print(url)
    r = requests.get(url)
    return r.json()

In [7]:
def get_raw_weather_forecast(office, gridX, gridY):
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'
    print(url)
    r = requests.get(url)
    return r.json()

In [8]:
#forecast_12_hour = get_weather_forecast('LOX', 160, 48, '12_hour')
#forecast_7day_hourly = get_weather_forecast('LOX', 160, 48, '7day_hourly')
#forecast_7day_raw = get_weather_forecast('LOX', 160, 48, '7day_grid')

In [9]:
#forecast_7day_hourly

In [10]:
ct_hourly_forecast = get_hourly_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
ct_hourly_forecast

https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-121.9122352, 37.341288],
    [-121.9067053, 37.3194264],
    [-121.879278, 37.323810699999996],
    [-121.8848026, 37.345672799999996],
    [-121.9122352, 37.341288]]]},
 'properties': {'updated': '2024-04-02T20:33:27+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-04-03T04:00:35+00:00',
  'updateTime': '2024-04-02T20:33:27+00:00',
  'validTimes': '2024-04-02T14:00:00+00:00/P7DT14H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 24.9936},
  'periods': [{'number': 1,
    'name': 'Tonight',
    'startTime': '2024-04-02T21:00:00-07:00',
    'endTime': '2024-04-03T06:00:00-07:0

In [60]:
import numpy as np
def create_hourly_forecast_df(json_forecast):
    # grab each hourly entry in the forecast
    forecast = json_forecast['properties']['periods']
    forecast_df = pd.DataFrame(forecast)
    
    # convert startTime to UTC datetime and set as index
    forecast_df['time'] = pd.to_datetime(forecast_df['startTime'], utc=True)
    forecast_df = forecast_df.set_index('time')
    
    # convert temperature to degF and degC
    forecast_df.rename(columns = {'temperature':'temperature_degF'}, inplace = True) 
    forecast_df['temperature_degC'] = forecast_df['temperature_degF'].apply(lambda x: np.round((x - 32) * 5.0/9.0, 2))
    
    # convert dewpoint temperature to degF and degC
    forecast_df['dewpoint_degC'] = forecast_df['dewpoint'].apply(lambda x: np.round(x['value'], 2))
    forecast_df['dewpoint_degF'] = forecast_df['dewpoint_degC'].apply(lambda x: np.round(x * 9.0/5.0 + 32, 2))
    
    # extract value from windSpeed column and convert to int
    forecast_df['windSpeed_mph'] = forecast_df['windSpeed'].apply(lambda x: int(x.split(" mph")[0]))
    
    # extract value from json formatted columns
    forecast_df['probabilityOfPrecipitationPercent'] = forecast_df['probabilityOfPrecipitation'].apply(lambda x: x['value'])
    forecast_df['relativeHumidityPercent'] = forecast_df['relativeHumidity'].apply(lambda x: x['value'])
    
    # drop columns that are no longer needed
    return forecast_df.drop(columns=['startTime', 'endTime', 'windSpeed', 'number', 'name', 'detailedForecast', 'dewpoint', 'probabilityOfPrecipitation', 'relativeHumidity', 'temperatureTrend', 'temperatureUnit'])

In [61]:
ct_hourly_forecast_df = create_hourly_forecast_df(ct_hourly_forecast)
ct_hourly_forecast_df

,isDaytime,temperature_degF,windDirection,icon,shortForecast,temperature_degC,dewpoint_degC,dewpoint_degF,windSpeed_mph,probabilityOfPrecipitationPercent,relativeHumidityPercent
time,,,,,,,,,,,
2024-04-03 02:00:00+00:00,False,68,WSW,"https://api.weather.gov/icons/land/night/few,0...",Mostly Clear,20.00,6.67,44.01,10,0,43
2024-04-03 03:00:00+00:00,False,66,W,"https://api.weather.gov/icons/land/night/few,0...",Mostly Clear,18.89,7.78,46.00,5,0,49
2024-04-03 04:00:00+00:00,False,63,W,"https://api.weather.gov/icons/land/night/few,0...",Mostly Clear,17.22,7.78,46.00,5,0,54
2024-04-03 05:00:00+00:00,False,61,W,"https://api.weather.gov/icons/land/night/few,0...",Mostly Clear,16.11,7.78,46.00,5,0,59
2024-04-03 06:00:00+00:00,False,59,ENE,"https://api.weather.gov/icons/land/night/few,0...",Mostly Clear,15.00,7.22,45.00,5,0,61
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-09 09:00:00+00:00,False,48,NE,"https://api.weather.gov/icons/land/night/few,1...",Mostly Clear,8.89,6.11,43.00,5,1,82
2024-04-09 10:00:00+00:00,False,47,NE,"https://api.weather.gov/icons/land/night/few,1...",Mostly Clear,8.33,5.56,42.01,5,1,83
2024-04-09 11:00:00+00:00,False,47,NE,"https://api.weather.gov/icons/land/night/few,1...",Mostly Clear,8.33,5.56,42.01,5,1,83
